In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

# WordCloud and matplotlib for visualization
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## TO DO

- assigning keywords to missing keyword labels - DONE
- remove symbols (#, ".. - DONE
- remove links - DONE
- validation
- make slides - IN PROCESS
- TFIDF, LSA, LSTM / RNNs, the list is long
- try different classifiers
- reading discussion board for inspiration


In [13]:
train = pd.read_csv("preprocessed_train_data.csv", index_col = 0)
test = pd.read_csv("preprocessed_test_data.csv", index_col = 0)

trained_tweets = train['keyword']+train['text']
test_tweets = test['keyword']+test['text']

In [14]:
trained_tweets.head()

0    earthquak  deed reason earthquak may allah for...
1     forest fire  forest fire near la rang ask canada
2    evacu  resid ask shelter place notifi offic ev...
3    wildfir  peopl receiv wildfir evacu order cali...
4    wildfir  got sent photo rubi alaska smoke wild...
dtype: object

In [15]:
test_tweets.head()

0                      crash  happen terribl car crash
1    earthquak  heard earthquak differ citi stay sa...
2    forest fire  forest fire spot pond gees flee a...
3            apocalyps  apocalyps light spokan wildfir
4          typhoon  typhoon soudelor kill china taiwan
dtype: object

# Encoding and Vectorizers

In [16]:
# import category_encoders as ce

# # Target encoding
# features = ['keyword']
# encoder = ce.TargetEncoder(cols=features)
# encoder.fit(train[features],train['target'])

# train = train.join(encoder.transform(train[features]).add_suffix('_target'))
# test = test.join(encoder.transform(test[features]).add_suffix('_target'))

In [17]:
X_train, X_test, y_train, y_test = train_test_split(trained_tweets, train['target'].values, 
        train_size = 0.80, test_size = 0.20, random_state = 12, shuffle=True) # We have split train-test dataset using 80:20 ratio

# TFIDF

In [18]:
#precompute vectorized representations
word_vectorizer = TfidfVectorizer(
    analyzer='word',
    stop_words='english',
    ngram_range=(1, 3),
    lowercase=True,
    min_df=5,
    max_features=30000)

char_vectorizer = TfidfVectorizer(
    analyzer='char',
    stop_words='english',
    ngram_range=(3, 6),
    lowercase=True,
    min_df=5,
    max_features=50000)

vectorizer = FeatureUnion([('word_vectorizer', word_vectorizer),  ('char_vectorizer', char_vectorizer)])
vectorizer.fit(X_train)

X_train_vectors = vectorizer.transform(X_train).toarray()
X_test_vectors = vectorizer.transform(X_test).toarray()
print(X_train_vectors.shape, X_test_vectors.shape)

#X_train_text = X_train.tolist()
#X_test_text = X_test.tolist()

(6090, 48864) (1523, 48864)


In [19]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [ ]:
from sklearn.model_selection import GridSearchCV

scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train_vectors, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df